In [3]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})
# np.set_printoptions(linewidth=200, formatter={'float': '{:.1e}'.format})

In [75]:
nH = 1e-9
fF = 1e-15
GHz = 1e9

In [76]:
h    = 6.626e-34
GHz = 1e9
e0   = 1.602e-19
Φ_0 = h/(2*e0)

In [77]:
# Circuit parameters
Csh = 15
C   = 15
CJ  = 3
Lq  = 25
Lr  = 10
Δ   = 0.1
EJ = 10
l = Lq*(Lq+4*Lr) - 4*Δ**2

In [81]:
Cf = C/2 + Csh + CJ
Lf = l/(Lq+4*Lr)

C_f = Cf * fF
L_f = Lf * nH
E_J = EJ * GHz

In [96]:
fluxonium = sq_ext.KIT_fluxonium()

In [103]:
q = fluxonium.charge_op(0) / C_f
qq = fluxonium.charge_op(0)**2 / C_f
Φ = fluxonium.flux_op(0) / L_f 
# H = 0.5 * C_f**-1 * q**2 + 0.5 * L_f**-1 * Φ**2 + E_J * Φ / (Φ_0/(2*np.pi))  * np.sqrt(h)
H = fluxonium.hamiltonian()                                                                                                             

In [104]:
sq_ext.H_eff_p1(H,H,2)

array([[6.707030428456, -0.000000000000],
       [-0.000000000000, 8.652527257331]])

In [105]:
sq_ext.H_eff_p1(H,Φ,2)

array([[-0.000000000000, 0.244590355252],
       [0.244590355252, 0.000000000000]])

In [100]:
sq_ext.H_eff_p1(H,q,2)

array([[ 5.25958962e-15-2.05339890e-30j, -1.33335600e-13-7.47444657e+01j],
       [-1.33335600e-13+7.47444657e+01j, -6.18153888e-13+1.27239777e-28j]])

In [106]:
sq_ext.H_eff_p1(H,qq,2)

array([[1.326936927337, 0.000000000000],
       [0.000000000000, 5.179339181612]])